# 0. SUMO
The aim of this notebook is to make a quick guide on how to simulate traffic using "Simulation of Urban Mobility" (SUMO) of our dataset.

## Useful imports

In [2]:
from xml.dom import minidom
import os 
import datetime

import numpy as np
import pandas as pd

## How to setup SUMO

To install SUMO : https://sumo.dlr.de/docs/Installing/index.html#macos

A Hello SUMO tutorial that is enough to understand how to set up the simulation : https://sumo.dlr.de/docs/Tutorials/Hello_SUMO.html

To simulate urban mobility, we have to define some files :
- hello.nod.xml, nodes are junctions that have locations (x,y) coordinates, describing distance in meters to origin.
- hello.edg.xml, edges connect nodes and have a direction (from node 1 to 2 for ex.)

From these 2 files, a hello.net.xml is generated to create road network by typing in the terminal (make sure to be in the repository src) : 

_netconvert --node-files=sumo-files/shello.nod.xml --edge-files=sumo-files/hello.edg.xml --output-file=sumo-files/hello.net.xml_

- hello.rou.xml, we define 'vType' that have basic properties such as length, acceleration, maximum speed and id. Then from these vType we define 'vehicle' its depart time and its route.

- hello.settings.xml, when simulating with SUMO it's useful to add a gui-settings file, so we don't have to change the settings after starting the program. 

- hello.sumocfg, where we put together hello.net.xml, hello.rou.xml and hello.settings.xml so we can do the simulation. 

To start the simulation : 

- Open the application XQuartz
- In the terminal : sumo-gui -c hello.sumocfg

More info on properties of vehicles https://sumo.dlr.de/docs/Definition_of_Vehicles%2C_Vehicle_Types%2C_and_Routes.html#depart

### Load data

For this little warmout with SUMO, we use a small dataset provided by our lab.

In [3]:
DATA_PATH = "../data/extract.txt"

cols_map = {
    "DTS": "depart",
    "LANE": "route",
    "SPD": "departSpeed",
    "TYPE": "type",
    "LENTH": "length"
}

columns = list(cols_map.values())

df = pd.read_csv(DATA_PATH, usecols=cols_map.keys())
df.rename(columns=cols_map, inplace=True)
df.depart = pd.to_datetime(df.depart, format="%Y%m%dT%H%M%S")

df.head()

,depart,route,departSpeed,length,type
0,2000-01-04 00:05:57,1,101,835,11
1,2000-01-04 00:28:24,1,83,640,11
2,2000-01-04 00:34:57,1,84,750,11
3,2000-01-04 01:14:14,1,89,832,11
4,2000-01-04 02:25:26,1,104,950,12


#### Calculate mean of length of every type of vehicle

We calculate the mean of length of every type of vehicle to simplify the simulation in order to have a unique length for every type when defining the vehicle in hello.rou.xml

In [5]:
#Convert type to string value for xml file
df['type'] = df['type'].astype(str)
means = []
types = df['type'].unique()

#Find means length of every type of vehicule
for val in types:
    means.append(df.loc[df['type'] == val]['length'].mean())

means = [str(x) for x in means]
routes = []

#Create dictionary with every vType
for i in range(len(means)):
    routes.append({'vType': {"id": types[i], "length": means[i]}})

#### Create a dict of all vehicles

We create a list of every vehicles of our dataset with its crossing time to pass on hello.rou.xml file

In [9]:
#the first car departs at t = 0
departs = [0]
for i in range(len(df)-1):
    t1 = df['depart'][i].timestamp()
    t2 = df['depart'][i+1].timestamp()
    delay = t2-t1
    departs.append(departs[i] + delay)
    
departs = [str(x) for x in departs]

for i in range(len(df)):
    routes.append({'vehicle': {"depart": departs[i], "type": df.iloc[i]['type'], "id": str(i), "route": "route0"}})

**Create and fill hello.rou.xml file**

In [10]:
routes_file = minidom.Document()
  
xml = routes_file.createElement('routes') 
routes_file.appendChild(xml)
  
route = routes_file.createElement('route')
route.setAttribute('id', 'route0')
route.setAttribute('edges', '1to2 out')

xml.appendChild(route)

for i in range(len(routes)):
    if 'vType' in routes[i]:
        key = 'vType'
        vType = routes_file.createElement(key)
        vType.setAttribute('id', routes[i][key]['id'])
        vType.setAttribute('length', routes[i][key]['length'])
        xml.appendChild(vType)
    else:
        key = 'vehicle'
        vehicle = routes_file.createElement(key)
        vehicle.setAttribute('depart', routes[i][key]['depart'])
        vehicle.setAttribute('type', routes[i][key]['type'])
        vehicle.setAttribute('id', routes[i][key]['id'])
        vehicle.setAttribute('route', 'route0')
        xml.appendChild(vehicle)

xml_str = routes_file.toprettyxml(indent ="\t") 
  
save_path_file = "sumo-files/hello.rou.xml"
  
with open(save_path_file, "w") as f:
    f.write(xml_str) 

Then, go back to the part on "how to setup SUMO" !